In [15]:
import fire
import os
import requests
import pandas as pd
from dotenv import load_dotenv
load_dotenv()
import numpy as np
import alpaca_trade_api as alpaca
import questionary

from qualifier.utils.alpaca_api import alpaca_func
from Modules.CleanData import get_data
# from qualifier.utils.SQL import add_new_table

# alpaca_func(key=alpaca_key, secret=alpaca_secret, START_DATE=START_DATE, END_DATE=END_DATE)
# add_new_table(data=a)



In [16]:
alpaca_key = os.getenv("ALPACA_API_KEY")
alpaca_secret = os.getenv("ALPACA_SECRET_KEY")

start_date = "2022-01-01"
end_date = "2022-05-30"

In [17]:
alpaca_func(alpaca_key, alpaca_secret, start_date, end_date)

Successfully gathered data from API.


In [18]:
df = get_data('09-14-2022')
df.head()

,Symbol,Price,Type,Strike,Exp Date,DTE,Bid,Midpoint,Ask,Last,Volume,Open Int,OI Chg,IV,Time
1907,AA,44.81,Put,43.0,2022-09-16,2.0,0.40,0.43,0.45,0.41,600.0,188.0,-19,83.62%,2022-09-14
1839,AA,44.81,Put,45.0,2022-09-16,2.0,1.16,1.20,1.24,1.19,1360.0,2098.0,-27,82.53%,2022-09-14
155,AAL,14.03,Call,14.0,2022-09-16,2.0,0.26,0.27,0.27,0.26,10637.0,22188.0,"+2,212",58.91%,2022-09-14
159,AAL,14.03,Put,14.0,2022-09-16,2.0,0.23,0.24,0.25,0.24,1931.0,16516.0,"+2,156",61.74%,2022-09-14
1237,AAPL,155.31,Call,140.0,2022-09-16,2.0,15.25,15.40,15.55,15.54,674.0,21206.0,-399,75.41%,2022-09-14
